<a href="https://colab.research.google.com/github/schmidchr/3Dimage_DataSciWorkshop/blob/master/Final_code_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# first you need to download raw.zip file from here: https://drive.google.com/file/d/1Zh6hTVQpHsug_xrPyAKQ-OD1VhQHJtBX/view?usp=sharing
# the raw.zip file should be placed on your Google drive 
from google.colab import drive
# to see your drive and other files 
drive.mount('/content/drive') 
# you need to follow the link and get the code each time you run the code

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [16]:
# create directories for notebook in Google Drive web interface

# find downloaded raw.zip file
#!ls  /content/drive/My\ Drive/Colab\ Notebooks/US_segmentation/raw/ 
!ls  /content/
!find "/" -name raw.zip


drive  sample_data
/content/drive/My Drive/Colab Notebooks/US_segmentation/raw/raw.zip


In [19]:
# unzip your raw.zip if have not done already
!unzip "/content/drive/My Drive/Colab Notebooks/US_segmentation/raw/raw.zip" # e.g."/content/drive/raw.zip"

# in your folder you should have
# ./raw/
# ./raw/train/ 
# ./raw/test/
# Final_code_segmentation.ipynb


Archive:  /content/drive/My Drive/Colab Notebooks/US_segmentation/raw/raw.zip
   creating: test/
  inflating: test/1.tif              
  inflating: test/10.tif             
  inflating: test/100.tif            
  inflating: test/1000.tif           
  inflating: test/1001.tif           
  inflating: test/1002.tif           
  inflating: test/1003.tif           
  inflating: test/1004.tif           
  inflating: test/1005.tif           
  inflating: test/1006.tif           
  inflating: test/1007.tif           
  inflating: test/1008.tif           
  inflating: test/1009.tif           
  inflating: test/101.tif            
  inflating: test/1010.tif           
  inflating: test/1011.tif           
  inflating: test/1012.tif           
  inflating: test/1013.tif           
  inflating: test/1014.tif           
  inflating: test/1015.tif           
  inflating: test/1016.tif           
  inflating: test/1017.tif           
  inflating: test/1018.tif           
  inflating: test/1019.tif   

In [20]:
# version of python
from __future__ import print_function
# load library (module) for system. Interface to the operating system
import os
# load library for working with arrays
import numpy as np
# load library for saving and reading images
from skimage.io import imsave, imread
# for ploting
import matplotlib.pyplot as plt
# path to the data directory
data_path = '/content/drive/My Drive/Colab Notebooks/US_segmentation/raw/'
# sizi of the raw images
image_rows = 420
image_cols = 580
os.chdir("/content/drive/My Drive/Colab Notebooks/US_segmentation/")
! pwd

/content/drive/My Drive/Colab Notebooks/US_segmentation


In [0]:
# creat a funciton that loads training images
def create_train_data():
    # path to the images /raw/train/
    train_data_path = os.path.join(data_path, 'train')
    # get names of the images
    images = os.listdir(train_data_path)
    print(images)
    # total = number of images/2 beacause we have masks in the folder
    total = len(images) / 2
    print('Number of images that we load for the training', total)
    # Reserve memory. imgs - array of images ina stack 
    imgs = np.ndarray((total, image_rows, image_cols), dtype=np.uint8)
    # and corresponding masks
    imgs_mask = np.ndarray((total, image_rows, image_cols), dtype=np.uint8)

    i = 0
    print('-'*30) #makes a long hyphens string
    print('Creating training images...')
    print('-'*30) 
    # loop through the images
    for image_name in images:
        # if mask is a part of the image names than skip the loop
        if 'mask' in image_name:
            continue
        # crates a names for the mask file (string)
        image_mask_name = image_name.split('.')[0] + '_mask.tif'
        # read the image
        img = imread(os.path.join(train_data_path, image_name), as_grey=True)
        # read the mask image
        img_mask = imread(os.path.join(train_data_path, image_mask_name), as_grey=True)
        # convert grey scale image to an array
        img = np.array([img])
        img_mask = np.array([img_mask])
        # put it in the 3D array
        imgs[i] = img
        imgs_mask[i] = img_mask

        if i % 100 == 0:
            print('Done: {0}/{1} images'.format(i, total))
        i += 1
    print('Loading done.')
    
    img1=imgs[1];
    print('Size of the image',img1.shape)
    imgplot = plt.imshow(img1)
    plt.show()
    # save array with images. It will help us to access them alter easier
    np.save('imgs_train.npy', imgs)
    np.save('imgs_mask_train.npy', imgs_mask)
    print('Saving to .npy files done.')

In [22]:
# function that loads 3D arrays of images
def load_train_data():
    imgs_train = np.load('imgs_train.npy')
    imgs_mask_train = np.load('imgs_mask_train.npy')
#     img=imgs_train[1][:][:];
#     print('Size of the image',imgs_train.shape)
#     imgplot = plt.imshow(img)
#     plt.show()
    return imgs_train, imgs_mask_train

# fucntion that loads the test images and saves as a 3D array. See create_train_data()
def create_test_data():
    train_data_path = os.path.join(data_path, 'test')
    images = os.listdir(train_data_path)
    total = len(images)

    imgs = np.ndarray((total, image_rows, image_cols), dtype=np.uint8)
    imgs_id = np.ndarray((total, ), dtype=np.int32)

    i = 0
    print('-'*30)
    print('Creating test images...')
    print('-'*30)
    for image_name in images:
        img_id = int(image_name.split('.')[0])
        img = imread(os.path.join(train_data_path, image_name), as_grey=True)

        img = np.array([img])

        imgs[i] = img
        imgs_id[i] = img_id

        if i % 100 == 0:
            print('Done: {0}/{1} images'.format(i, total))
        i += 1
    print('Loading done.')
    # show one image
    img1=imgs[1];
    print('Size of the image',img1.shape)
    imgplot = plt.imshow(img1)
    plt.show()
    np.save('imgs_test.npy', imgs)
    np.save('imgs_id_test.npy', imgs_id)
    print('Saving to .npy files done.')


def load_test_data():
    imgs_test = np.load('imgs_test.npy')
    imgs_id = np.load('imgs_id_test.npy')
    return imgs_test, imgs_id

# this the main function
if __name__ == '__main__':#__variable_name__ but hidden
    create_train_data()
    create_test_data()

FileNotFoundError: ignored

In [0]:
######### train.py - that is where we train our network
###############################################################

In [0]:
#from __future__ import print_function

#import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from keras.models import Model 
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam 
from keras.callbacks import ModelCheckpoint
from keras import backend as K # import form Keras as backend and change the name to K

#os.chdir("/content/drive/My Drive/Colab Notebooks/US_segmentation/")
#! pwd

#from data import load_train_data, load_test_data #load 2 functions from data.py

K.set_image_data_format('channels_last')  # TF dimension ordering in this code 
# new size of images. We want to downsample them
img_rows = 96
img_cols = 96
# parameter
smooth = 1.
print(imsave)
print(Conv2D)
print(Input)

In [0]:
# we want to maximize dice coefficient
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)# it converts from ND to 1D
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

#we want to minimize the loss (error)
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def get_unet():
    # use functions from Keras, to deconvolute the images in 5 different ways. Input layer for the network
    inputs = Input((img_rows, img_cols, 1))# 96x96x1. 3D Tensor
    print(inputs)            
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs) #shape=(?, 96, 96, 32) dtype=float32>
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)#shape=(?, 48, 48, 32) dtype=float32>

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)#shape=(?, 48, 48, 64) dtype=float32>
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)#shape=(?, 24, 24, 64)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    
    # Reverse the convolution to obtain the initial image               
    #up6: shape=(?, 12, 12, 512) or TensorShape([Dimension(None), Dimension(12), Dimension(12), Dimension(512)])
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

    return model

# function that has an input 3D image array. It downsamples the input.
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)

    imgs_p = imgs_p[..., np.newaxis]#?????? check the structure
    return imgs_p


def train_and_predict():
    print('-'*30)
    print('Loading and preprocessing train data...')
    print('-'*30)
    # load the data
    imgs_train, imgs_mask_train = load_train_data()
    # downsample images with their masks
    imgs_train = preprocess(imgs_train)
    imgs_mask_train = preprocess(imgs_mask_train)
    #  convert ot float format
    imgs_train = imgs_train.astype('float32')
    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization
    # imgs_train = imgs_train - mean. This way we account for background variations
    imgs_train -= mean
    # imgs_train = imgs_train/std. This way we account for large intensity variations between images.
    imgs_train /= std

    imgs_mask_train = imgs_mask_train.astype('float32')
    imgs_mask_train /= 255.  # scale masks to [0, 1]

    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    # build U net (neural network) model
    model = get_unet()
    # to store the itteration in weights.h5
    model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)

    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    # train neural network. 
    model.fit(imgs_train, imgs_mask_train, batch_size=32, nb_epoch=20, verbose=1, shuffle=True,
              validation_split=0.2,
              callbacks=[model_checkpoint])

    print('-'*30)
    print('Loading and preprocessing test data...')
    print('-'*30)
    # imgs_id_test is a raw with int that corresponds to the filenames
    imgs_test, imgs_id_test = load_test_data()
    imgs_test = preprocess(imgs_test)

    imgs_test = imgs_test.astype('float32')
    imgs_test -= mean
    imgs_test /= std

    print('-'*30)
    print('Loading saved weights...')
    print('-'*30)
    model.load_weights('weights.h5')

    print('-'*30)
    print('Predicting masks on test data...')
    print('-'*30)
    imgs_mask_test = model.predict(imgs_test, verbose=1)
    np.save('imgs_mask_test.npy', imgs_mask_test)

    print('-' * 30)
    print('Saving predicted masks to files...')
    print('-' * 30)
    pred_dir = 'preds'
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dir)
    # iterating the image and the index    
    # zip connects to list 
    # imgs_id_test - a filename
    for image, image_id in zip(imgs_mask_test, imgs_id_test):
        image = (image[:, :, 0] * 255.).astype(np.uint8)
        imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)

if __name__ == '__main__':
    train_and_predict()

In [0]:
####### submission.py - the results should be converted to the ones required by submission
#################################################################################

In [0]:
#from __future__ import print_function
#import os
#import numpy as np
from skimage.transform import resize
#from data import image_cols, image_rows


def prep(img):
    img = img.astype('float32')
    img = (img > 0.5).astype(np.uint8)  # threshold
    img = resize(img, (image_cols, image_rows), preserve_range=True)
    return img


def run_length_enc(label):
    from itertools import chain
    x = label.transpose().flatten()
    y = np.where(x > 0)[0]
    if len(y) < 10:  # consider as empty
        return ''
    z = np.where(np.diff(y) > 1)[0]
    start = np.insert(y[z+1], 0, y[0])
    end = np.append(y[z], y[-1])
    length = end - start
    res = [[s+1, l+1] for s, l in zip(list(start), list(length))]
    res = list(chain.from_iterable(res))
    return ' '.join([str(r) for r in res])


def submission():
    #from data import load_test_data
    imgs_test, imgs_id_test = load_test_data()
    imgs_test = np.load('imgs_mask_test.npy')

    argsort = np.argsort(imgs_id_test)
    imgs_id_test = imgs_id_test[argsort]
    imgs_test = imgs_test[argsort]

    total = imgs_test.shape[0]
    ids = []
    rles = []
    for i in range(total):
        img = imgs_test[i, 0]
        img = prep(img)
        rle = run_length_enc(img)

        rles.append(rle)
        ids.append(imgs_id_test[i])

        if i % 100 == 0:
            print('{}/{}'.format(i, total))

    first_row = 'img,pixels'
    file_name = 'submission.csv'

    with open(file_name, 'w+') as f:
        f.write(first_row + '\n')
        for i in range(total):
            s = str(ids[i]) + ',' + rles[i]
            f.write(s + '\n')


if __name__ == '__main__':
    submission()
